# Strands Agents를 위한 노트북 환경 준비

이 섹션에서는 Strands 프레임워크를 사용하여 에이전트를 생성하는 데 필요한 종속성으로 노트북 환경을 준비합니다.

# 사전 요구사항

1. AWS 호스트 이벤트의 일부가 아닌 자체 계정에서 노트북을 실행하는 경우, [Self Paced](https://catalog.us-east-1.prod.workshops.aws/workshops/0bf501fc-357d-43b3-b758-b44f9b0e7d49/en-US/introduction/access-and-navigate-aws-account/self-paced) 워크샵의 **배포** 지침을 따라 계정에 다음 종속성을 설정하세요:
    - 네트워킹 인프라 (VPC, 서브넷 등)
    - Amazon Redshift 데이터베이스
    - Amazon Bedrock Knowledge Base
2. 다음 모델에 대한 Bedrock 모델 액세스를 구성하세요:
    - Amazon
        - Titan Embeddings G1 - Text
    - Anthropic
        - Anthropic Claude 4.5 Sonnet
3. 노트북 실행 역할에 다음 관리형 정책이 있는지 확인하세요:
    - AmazonBedrockFullAccess
    - AmazonRedshiftQueryEditor
    - AmazonS3FullAccess
    - AmazonSageMakerFullAccess
    - AWSLambda_FullAccess
    - AWSStepFunctionsFullAccess
    - IAMFullAccess
    - AWSCodeBuildAdminAccess
4. 단백질 디자인 노트북의 경우, 추가 AWS HealthOmics 권한이 이 설정 노트북에 의해 자동으로 추가됩니다.

# 환경 설정

#### 필요한 모든 패키지를 설치하려면 아래 pip 명령을 실행하세요

In [ ]:
%pip install boto3 awscli botocore termcolor sagemaker pytrials streamlit --quiet

아래에 출력된 boto3 버전이 **1.37.1** 이상인지 확인하세요.

In [ ]:
%pip show boto3

#### Python 라이브러리 가져오기

In [ ]:
# Standard Python libraries
import boto3
import sagemaker

# Import needed functions to create agent
from utils.bedrock_agent_helper import AgentsForAmazonBedrock
from utils.role_policy_helper import SageMakerRolePolicyChecker

#### 에이전트 생성에 필요한 SageMaker 역할 및 계정 정보 추출

In [ ]:
# boto3 세션
sts_client = boto3.client('sts')
session = boto3.session.Session()

# 계정 정보
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
print(f"계정 ID: {account_id}")
print(f"리전: {region}")

# SageMaker 세션 및 실행 역할 가져오기
sagemaker_session = sagemaker.Session()
role = sagemaker_session.get_caller_identity_arn()
print(f"SageMaker 실행 역할: {role}")

#### 노트북 실행에 필요한 정책이 SageMaker 역할에 있는지 확인

In [ ]:
# 필수 정책 확인 및 추가
role_checker = SageMakerRolePolicyChecker()
role_name = role.split('/')[-1]  # ARN에서 역할 이름 추출

try:
    # 기존 정책 확인
    role_checker.check_policies(role)
    print("모든 필수 관리형 정책이 연결되어 있습니다!")
except Exception as e:
    print(f"누락된 정책: {e}")
    print("누락된 정책을 수동으로 연결하거나 관리자에게 문의하세요.")



#### 워크샵에서 사용될 주요 AWS 서비스에 대한 권한 테스트

In [ ]:
# 주요 AWS 서비스 권한 테스트
from termcolor import colored
import json

def test_service_permission(client_name, test_operation, operation_params=None):
    """서비스 권한 테스트 도우미 함수"""
    try:
        client = boto3.client(client_name, region_name=region)
        
        # 테스트 작업 실행
        if operation_params:
            response = getattr(client, test_operation)(**operation_params)
        else:
            response = getattr(client, test_operation)()
            
        return True, "권한 확인됨"
    except client.exceptions.ResourceNotFoundException:
        # 리소스가 없는 것은 권한 문제가 아님
        return True, "권한 확인됨 (리소스 없음)"
    except Exception as e:
        error_msg = str(e)
        if "AccessDenied" in error_msg or "UnauthorizedOperation" in error_msg:
            return False, f"권한 거부: {error_msg[:100]}..."
        elif "ResourceNotFound" in error_msg or "does not exist" in error_msg:
            return True, "권한 확인됨 (리소스 없음)"
        else:
            # 다른 에러는 권한과 관련 없을 수 있음
            return True, f"권한 확인됨 (기타: {error_msg[:50]}...)"

# 테스트할 서비스 목록
services_to_test = [
    {
        "name": "Amazon Bedrock (Knowledge Base)",
        "client": "bedrock-agent",
        "operation": "list_knowledge_bases",
        "params": {"maxResults": 1}
    },
    {
        "name": "Amazon Bedrock (Agents/AgentCore)",
        "client": "bedrock-agent",
        "operation": "list_agents",
        "params": {"maxResults": 1}
    },
    {
        "name": "Amazon Bedrock (Models)",
        "client": "bedrock",
        "operation": "list_foundation_models",
        "params": None
    },
    {
        "name": "Amazon S3",
        "client": "s3",
        "operation": "list_buckets",
        "params": None
    },
    {
        "name": "AWS Lambda",
        "client": "lambda",
        "operation": "list_functions",
        "params": {"MaxItems": 1}
    },
    {
        "name": "Amazon Redshift",
        "client": "redshift",
        "operation": "describe_clusters",
        "params": None
    }
]

print("=" * 60)
print("AWS 서비스 권한 테스트 결과")
print("=" * 60)

all_passed = True
for service in services_to_test:
    success, message = test_service_permission(
        service["client"],
        service["operation"],
        service["params"]
    )
    
    if success:
        status = colored("✓ 통과", "green")
    else:
        status = colored("✗ 실패", "red")
        all_passed = False
    
    print(f"{status} - {service['name']}")
    if not success:
        print(f"     {message}")

print("=" * 60)

if all_passed:
    print(colored("✓ 모든 서비스 권한 테스트 통과!", "green"))
    print("워크샵을 진행할 준비가 완료되었습니다.")
else:
    print(colored("⚠ 일부 권한이 누락되었습니다.", "yellow"))
    print("위에서 실패한 서비스에 대한 권한을 확인하고 추가해주세요.")